In [3]:
class BertModel(torch.nn.Module):

    def __init__(self):
        
        super(BertModel, self).__init__()
        unique_labels = {'B-ACTIVE',
 'B-ADULT',
 'B-AGE',
 'B-AMMO',
 'B-ATHLETIC',
 'B-BABY',
 'B-BIRTH',
 'B-BOOT',
 'B-BRAND',
 'B-CANOPY',
 'B-CUP',
 'B-DANCE',
 'B-DEPARTMENT',
 'B-DESIGN',
 'B-DISPLAY',
 'B-FASTENING',
 'B-FIT',
 'B-FORM',
 'B-GENDER',
 'B-HEEL',
 'B-INTEREST',
 'B-KIDS',
 'B-LEG',
 'B-LENGTH',
 'B-LENS',
 'B-LOCATION',
 'B-LOCK',
 'B-MATERIAL',
 'B-MECHANICAL',
 'B-MOBILITY',
 'B-NECKLINE',
 'B-OCCASION',
 'B-PADDING',
 'B-PATTERN',
 'B-RISE',
 'B-SANDAL',
 'B-SHAFT',
 'B-SHAPE',
 'B-SHOULDER',
 'B-SILHOUETTE',
 'B-SIZE',
 'B-SLEEVE',
 'B-SPECIALTY',
 'B-STYLE',
 'B-THEME',
 'B-TODDLER',
 'B-TOE',
 'B-TYPE',
 'B-VARIETY',
 'B-VISE',
 'B-WATCH',
 'COLOR',
 'DEPARTMENT',
 'GROUP',
 'HEIGHT',
 'I-AGE',
 'I-ATHLETIC',
 'I-BOOT',
 'I-BRAND',
 'I-CUP',
 'I-DANCE',
 'I-DESIGN',
 'I-DISPLAY',
 'I-FASTENING',
 'I-FIT',
 'I-HEEL',
 'I-LEG',
 'I-LENGTH',
 'I-LENS',
 'I-MATERIAL',
 'I-NECKLINE',
 'I-PADDING',
 'I-PATTERN',
 'I-RISE',
 'I-SHAFT',
 'I-SHAPE',
 'I-SHOULDER',
 'I-SILHOUETTE',
 'I-SIZE',
 'I-SLEEVE',
 'I-STYLE',
 'I-THEME',
 'I-TODDLER',
 'I-TOE',
 'I-TYPE',
 'I-VARIETY',
 'I-WATCH',
 'LEVEL',
 'MONTH',
 'MOVEMENT',
 'O',
 'SHOE',
 'SIZE',
 'STYLE',
 'TYPE',
 'USE'}
        self.bert = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(unique_labels))

    def forward(self, input_id, mask, label):

        output = self.bert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)

        return output



In [1]:
import os
import logging
import json
# import debugpy
# # Allows other computers to attach to debugpy on this IP address and port.
# debugpy.listen(('0.0.0.0', 5678))
# # Wait 30 seconds for a debugger to attach. If none attaches, the script continues as normal.
# debugpy.wait_for_client()
# print("Debugger attached...")
import torch
from transformers import BertTokenizerFast, BertForTokenClassification

C:\Users\madri\.conda\envs\mlenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
model = BertModel()
model.load_state_dict(torch.load("model22.pth"))
model.eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

BertModel(
  (bert): BertForTokenClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tru

In [ ]:
def align_word_ids(texts):
  
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):


    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    
    return sentence.split(" "), prediction_label
            
sentence, prediction_label = evaluate_one_text(model, "Congs bag red")

print(sentence)
print(" ".join(prediction_label))